<hr style="height:.9px;border:none;color:#333;background-color:#333;" />
<hr style="height:.9px;border:none;color:#333;background-color:#333;" />

<h2>Homework - Hyperparameter Tuning</h2>
<h4>DAT-5303 | Machine Learning</h4>

<br>
<hr style="height:.9px;border:none;color:#333;background-color:#333;" />
<hr style="height:.9px;border:none;color:#333;background-color:#333;" />

Team: TEAM 8

<hr style="height:.9px;border:none;color:#333;background-color:#333;" />
<br>
In this homework, your team is tasked with tuning the hyperparameters of two of the following model types:

* Regression Tree
* Random Forest
* Gradient Boosted Machine (GBM)

<h3>Step 1: Imports</h3>
Import the following packages and make sure to start replacing my comments with your own. Also, please tell me about which models you are tuning in the second cell.

In [1]:
# importing critical libraries
import pandas            as pd   # data science essentials
import matplotlib.pyplot as plt  # essential graphical output
import seaborn           as sns  # enhanced graphical output


# importing model types
import sklearn.linear_model                            # linear models (Linear Regression, etc)
from sklearn.tree     import DecisionTreeRegressor     # regression trees
from sklearn.ensemble import RandomForestRegressor     # random forest
from sklearn.ensemble import GradientBoostingRegressor # gbm


# importing machine learning tools
from sklearn.model_selection import train_test_split   # train-test split
from sklearn.tree            import plot_tree          # tree plots
from sklearn.model_selection import RandomizedSearchCV # Randomized Search CV


# loading data
housing = pd.read_excel('./__datasets/housing_feature_rich.xlsx')

# ***************************************************************************************************

# 1st Feauture " Square Feet Area" (Sum of selected areas)

housing['Sq_ft_area'] = housing['Gr_Liv_Area'] + housing['Total_Bsmt_SF'] + housing['Garage_Area']\
                     + housing['First_Flr_SF'] + housing['Second_Flr_SF'] + housing['Porch_Area']


# 2nd Feauture "House health" is the sum of scores of overall qual and overal cond in -
# - the proportion of 60:40 respectively

housing['House_health'] = (housing['Overall_Qual']) * 0.6 + (housing['Overall_Cond']) * 0.4


# 3rd Feature created with various datasets

# Step 1 placeholder variable creation " has Mas Vnr"

housing['has_Mas_Vnr']  = 0


# change values in the new feature columns

for index, value in housing.iterrows():

    # has_Mas_Vnr
    
    if housing.loc[index, 'Mas_Vnr_Area'] > 0:
        housing.loc[index, 'has_Mas_Vnr'] = 1

        
# Step 2 total bathrooms

housing['Total_Bath'] = housing['Full_Bath'] + (housing['Half_Bath'] / 2)


# Step 3  "Plus value" gathering all the created sub-features in we which have -
# - Fireplaces, has Mas Vnr, Garage cars, Total bath

housing['Plus_Value'] = housing['Fireplaces'] + housing['has_Mas_Vnr'] \
                       + housing['Garage_Cars'] + housing['Total_Bath']


# declaring sets of x-variables

# includes the various locations and the combinations team 8 made

x_variables = ['Garage_Cars', 'Total_Bsmt_SF', 'NridgHt', 'Kitchen_AbvGr', 
               'has_Second_Flr', 'Mas_Vnr_Area', 'has_Garage', 'Porch_Area',
               'NWAmes', 'OldTown', 'Edwards', 'Somerst', 'Fireplaces',
               'Second_Flr_SF', 'First_Flr_SF', 'has_Mas_Vnr',
               'CulDSac', 'Total_Bath', 'Crawfor', 'Garage_Area',
               'has_Porch', 'Sq_ft_area', 'House_health', 'Plus_Value']

# excludes the combinations the team 8 made but includes all the original comulns and locations

full_x      = ['Overall_Qual', 'Overall_Cond', 'Mas_Vnr_Area', 'Total_Bsmt_SF',
               'First_Flr_SF', 'Second_Flr_SF', 'Gr_Liv_Area', 'Full_Bath',
               'Half_Bath', 'Kitchen_AbvGr', 'TotRms_AbvGr', 'Fireplaces',
               'Garage_Cars', 'Garage_Area', 'Porch_Area', 'log_Lot_Area',
               'has_Second_Flr', 'has_Garage', 'has_Mas_Vnr', 'has_Porch',
               'Total_Bath', 'CulDSac', 'BrkSide', 'CollgCr', 'Crawfor',
               'Edwards', 'Gilbert', 'Mitchel', 'NWAmes', 'NridgHt', 'OldTown',
               'Sawyer', 'SawyerW', 'Somerst', 'Other_NH']

# includes the reduced x list and team 8's 3 engineered features: 

reduced_x   = ['Gr_Liv_Area', 'Full_Bath',
               'Kitchen_AbvGr', 'TotRms_AbvGr', 'Fireplaces',
               'Garage_Cars', 'Garage_Area', 'Porch_Area', 
               'log_Lot_Area', 'has_Second_Flr', 'has_Garage',
               'has_Mas_Vnr', 'has_Porch', 'Total_Bath', 'CulDSac',
               'Sq_ft_area', 'House_health', 'Plus_Value']

# preparing x-variables
# x_data = housing.loc[ : , reduced_x ] # x-variables (can change this)
x_data = housing.drop(['Sale_Price', 'log_Sale_Price'], axis = 1)

x_data = x_data[reduced_x]

# preparing y-variable
y_data = housing.loc[ : , 'Sale_Price']    # y-variable 
#y_data = housing.loc[ : , 'logSale_Price'] # y-variable

In [2]:
########################################
# plot_feature_importances
########################################
def plot_feature_importances(model, train, export = False):
    """
    Plots the importance of features from a CART model.
    
    PARAMETERS
    ----------
    model  : CART model
    train  : explanatory variable training data
    export : whether or not to export as a .png image, default False
    """
    
    # declaring the number
    n_features = x_train.shape[1]
    
    # setting plot window
    fig, ax = plt.subplots(figsize=(12,9))
    
    plt.barh(range(n_features), model.feature_importances_, align='center')
    plt.yticks(np.arange(n_features), train.columns)
    plt.xlabel("Feature importance")
    plt.ylabel("Feature")
    
    if export == True:
        plt.savefig('Tree_Leaf_50_Feature_Importance.png')

<br>

<hr style="height:.9px;border:none;color:#333;background-color:#333;" />
<h3>Step 2: Train-Test Split</h3>
Set up train-test split in the cell below. Set your <em>test_size</em> to 0.25 and your <em>random_state</em> to 219.

In [3]:
# train-test split, test size at 0.25 and random_state at 219
x_train, x_test, y_train, y_test = train_test_split(
                                                    x_data             ,
                                                    y_data             ,
                                                    test_size    = 0.25,
                                                    random_state = 219 )


<hr style="height:.9px;border:none;color:#333;background-color:#333;" />
<h3>Step 3: Check Available Hyperparameters</h3>
Call help(&nbsp;) on your selected model types to see which hyperparameters are available for tuning. Do NOT tune random_state, n_jobs or anything related to the intercept of a model. Use as many cells as needed.

In [4]:
# checking hyperparameters for Random Forest Regressos
help(RandomForestRegressor)

Help on class RandomForestRegressor in module sklearn.ensemble._forest:

class RandomForestRegressor(ForestRegressor)
 |  RandomForestRegressor(n_estimators=100, *, criterion='squared_error', max_depth=None, min_samples_split=2, min_samples_leaf=1, min_weight_fraction_leaf=0.0, max_features='auto', max_leaf_nodes=None, min_impurity_decrease=0.0, bootstrap=True, oob_score=False, n_jobs=None, random_state=None, verbose=0, warm_start=False, ccp_alpha=0.0, max_samples=None)
 |  
 |  A random forest regressor.
 |  
 |  A random forest is a meta estimator that fits a number of classifying
 |  decision trees on various sub-samples of the dataset and uses averaging
 |  to improve the predictive accuracy and control over-fitting.
 |  The sub-sample size is controlled with the `max_samples` parameter if
 |  `bootstrap=True` (default), otherwise the whole dataset is used to build
 |  each tree.
 |  
 |  Read more in the :ref:`User Guide <forest>`.
 |  
 |  Parameters
 |  ----------
 |  n_estimato

Available parameters:<br>
criterion: {"squared_error", "absolute_error", "poisson"},<br>             default="squared_error"<br>
max_depth : int, default=None<br>
min_samples_split : int or float, default=2<br>
min_samples_leaf : int or float, default=1<br>
random_state : int, RandomState instance or None, default=None<br>

In [5]:
help(DecisionTreeRegressor)

Help on class DecisionTreeRegressor in module sklearn.tree._classes:

class DecisionTreeRegressor(sklearn.base.RegressorMixin, BaseDecisionTree)
 |  DecisionTreeRegressor(*, criterion='squared_error', splitter='best', max_depth=None, min_samples_split=2, min_samples_leaf=1, min_weight_fraction_leaf=0.0, max_features=None, random_state=None, max_leaf_nodes=None, min_impurity_decrease=0.0, ccp_alpha=0.0)
 |  
 |  A decision tree regressor.
 |  
 |  Read more in the :ref:`User Guide <tree>`.
 |  
 |  Parameters
 |  ----------
 |  criterion : {"squared_error", "friedman_mse", "absolute_error",             "poisson"}, default="squared_error"
 |      The function to measure the quality of a split. Supported criteria
 |      are "squared_error" for the mean squared error, which is equal to
 |      variance reduction as feature selection criterion and minimizes the L2
 |      loss using the mean of each terminal node, "friedman_mse", which uses
 |      mean squared error with Friedman's improv

Available parameters: <br>
criterion: {"squared_error", "friedman_mse", "absolute_error", "poisson"}, <default="squared_error" <br>
splitter : {"best", "random"}, default="best" <br>
max_depth : int, default=None <br>
min_samples_split : int or float, default=2 <br>
min_samples_leaf : int or float, default=1 <br>
random_state : int, RandomState instance or None, default=None <br>

In [6]:
help(GradientBoostingRegressor)

Help on class GradientBoostingRegressor in module sklearn.ensemble._gb:

class GradientBoostingRegressor(sklearn.base.RegressorMixin, BaseGradientBoosting)
 |  GradientBoostingRegressor(*, loss='squared_error', learning_rate=0.1, n_estimators=100, subsample=1.0, criterion='friedman_mse', min_samples_split=2, min_samples_leaf=1, min_weight_fraction_leaf=0.0, max_depth=3, min_impurity_decrease=0.0, init=None, random_state=None, max_features=None, alpha=0.9, verbose=0, max_leaf_nodes=None, warm_start=False, validation_fraction=0.1, n_iter_no_change=None, tol=0.0001, ccp_alpha=0.0)
 |  
 |  Gradient Boosting for regression.
 |  
 |  GB builds an additive model in a forward stage-wise fashion;
 |  it allows for the optimization of arbitrary differentiable loss functions.
 |  In each stage a regression tree is fit on the negative gradient of the
 |  given loss function.
 |  
 |  Read more in the :ref:`User Guide <gradient_boosting>`.
 |  
 |  Parameters
 |  ----------
 |  loss : {'squared_er

<hr style="height:.9px;border:none;color:#333;background-color:#333;" />
<h3>Step 4: Model Development - Default Hyperparameters</h3>
Run each of your selected models using three (3) respective default hyperparameters that you feel would be good candidates for optimization, printing their R-Square values for the training and testing sets. Use as many cells as needed and make sure to label your default hyperparameters.

In [7]:
# specifying a model name
model_name = 'Pruned Random Forest'


# INSTANTIATING a random forest model with default values
model = RandomForestRegressor(n_estimators     = 100,
                              criterion        = 'squared_error',
                              max_depth        = 4,
                              min_samples_leaf = 25,
                              bootstrap        = True,
                              warm_start       = False,
                              random_state     = 219)


# FITTING the training data
model_fit = model.fit(x_train, y_train)


# PREDICTING based on the testing set
model_pred = model.predict(x_test)


# SCORING the results
model_train_score = model.score(x_train, y_train).round(4) # using R-square
model_test_score  = model.score(x_test, y_test).round(4)   # using R-square
model_gap         = abs(model_train_score - model_test_score).round(4)


# displaying results
print('Training Score :', model_train_score)
print('Testing Score  :', model_test_score)
print('Train-Test Gap :', model_gap)

Training Score : 0.8439
Testing Score  : 0.8231
Train-Test Gap : 0.0208


In [8]:
# specifying a model name
model_name = 'Pruned Regression Tree'

# INSTANTIATING a model object - CHANGE THIS AS NEEDED
model = DecisionTreeRegressor(max_depth        = 4   ,
                              min_samples_leaf = 25  ,
                              random_state     = 219,)


# FITTING to the training data
model_fit = model.fit(x_train, y_train)


# PREDICTING on new data
model_pred = model.predict(x_test)


# SCORING the results
model_train_score = model.score(x_train, y_train).round(4) # using R-square
model_test_score  = model.score(x_test, y_test).round(4)   # using R-square
model_gap         = abs(model_train_score - model_test_score).round(4)


# displaying results
print('Training Score :', model_train_score)
print('Testing Score  :', model_test_score)
print('Train-Test Gap :', model_gap)


Training Score : 0.8191
Testing Score  : 0.788
Train-Test Gap : 0.0311


<hr style="height:.9px;border:none;color:#333;background-color:#333;" />
<h3>Step 5: Hyperparameter Tuning Part I</h3><br>
Develop ranges for each of the hyperparameters you would like tune. Write a code to calculate how many models you are building (based on the combinations of hyperparameters you are tuning).
<br><br>
<strong>Requirement:</strong> You must tune at least three hyperparameters per model.<br>
<strong>Recommendation:</strong> Keep the number of models (i.e., iterations) you are building to:

* Less than 2,000 for Random Forest and GBM  (not including cross-validation)
* Less than 10,000 for all other model types (not including cross-validation)

Insert as many code cells as you need for the number of model types you are tuning.

In [9]:
# For Random Forest Regressor
# declaring a hyperparameter space
criterion_range = ["squared_error", "absolute_error", "poisson"]
depth_range     = range(1, 10, 1) # range(100, 1005, 50)
leaf_range      = range(1, 101, 25)

In [10]:
# For Decision Tree Regressor
# declaring a hyperparameter space
criterion_range = ["squared_error", "friedman_mse", "absolute_error", "poisson"]
splitter_range  = ["best", "random"]
depth_range     = range(1, 10, 1) # range(100, 1005, 50)
leaf_range      = range(1, 101, 25)

<hr style="height:.9px;border:none;color:#333;background-color:#333;" />
<h3>Step 6: Hyperparameter Tuning Part II</h3><br>
Create a hyperparameter grid for each of the models you are tuning. Note that this is the dictionary step we conducted in class. Use as many cells as needed for this task.

In [11]:
# For Random Forest Regressor
# Setting the param distributions
param_grid = {'criterion'        : criterion_range,
              'max_depth'        : depth_range,
              'min_samples_leaf' : leaf_range}

In [12]:
# For Decision Tree Regressor
# creating a hyperparameter grid
param_grid = {'criterion'        : criterion_range,
              'splitter'         : splitter_range,
              'max_depth'        : depth_range,
              'min_samples_leaf' : leaf_range}

<hr style="height:.9px;border:none;color:#333;background-color:#333;" />
<h3>Step 7: Hyperparameter Tuning Part III</h3><br>
a) Complete the remaining steps for hyperparameter tuning. While your code runs, observe the processing time it takes to tune. Use as many cells as needed for this task.

In [21]:
# declaring a hyperparameter space
criterion_range = ["squared_error", "friedman_mse", "absolute_error", "poisson"]
splitter_range  = ["best", "random"]
depth_range     = range(1, 10, 1)
leaf_range      = range(1, 101, 25)


# creating a hyperparameter grid
param_grid = {'criterion'        : criterion_range,
              'splitter'         : splitter_range,
              'max_depth'        : depth_range,
              'min_samples_leaf' : leaf_range}


# INSTANTIATING the model object without hyperparameters
tuned_tree = DecisionTreeRegressor(random_state = 219)


# RandomizedSearchCV object
tuned_tree_cv = RandomizedSearchCV(estimator             = tuned_tree,
                                   param_distributions   = param_grid,
                                   cv                    = 4,
                                   n_iter                = 500,
                                   random_state          = 219)


# FITTING to the FULL DATASET (due to cross-validation)
tuned_tree_cv.fit(x_data, y_data)


# printing the optimal parameters and best score
print("Tuned Parameters  :", tuned_tree_cv.best_params_)
print("Tuned Training AUC:", tuned_tree_cv.best_score_.round(decimals = 4))

/Users/purveshchitre/opt/anaconda3/lib/python3.9/site-packages/sklearn/model_selection/_search.py:292: UserWarning: The total space of parameters 288 is smaller than n_iter=500. Running 288 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


Tuned Parameters  : {'splitter': 'best', 'min_samples_leaf': 26, 'max_depth': 7, 'criterion': 'squared_error'}
Tuned Training AUC: 0.8248


**Test result of DecisionTreeRegressor:** <br>
Tuned Parameters  : <br>
'splitter' = 'best', <br>
'min_samples_leaf' = 26, <br>
'max_depth' = 7, <br>
'criterion' = 'squared_error' <br>
Tuned Training AUC = 0.8248
_______________________________________

In [14]:
# # declaring a hyperparameter space

# criterion_range = ["squared_error", "absolute_error", "poisson"]
# depth_range     = range(1, 10, 1)
# leaf_range      = range(1, 101, 25)


# # Setting the param distributions
# param_grid = {'criterion'        : criterion_range,
#               'max_depth'        : depth_range,
#               'min_samples_leaf' : leaf_range}


# # INSTANTIATING the model object without hyperparameters
# model = RandomForestRegressor(random_state = 219)



# # RandomizedSearchCV object
# model_cv = RandomizedSearchCV(estimator           = model,
#                               param_distributions = param_grid,
#                               n_iter              = 500,
#                               cv                  = 4,
#                               random_state        = 219)



# # FITTING to the FULL DATASET (due to cross-validation)
# model_cv.fit(x_data, y_data)



# # printing the optimal parameters and best score
# print(f"Tuned Parameters: {model_cv.best_params_}")
# print(f"Tuned R-Square:   {model_cv.best_score_.round(decimals = 4)}")



**Test result of RandomForestRegressor:** <br>
Tuned Parameters: <br>
'min_samples_leaf'= 1, <br>
'max_depth' = 9, <br>
'criterion' = 'absolute_error' <br>
Tuned R-Square =   0.8567602363718838
_______________________________________

<br>
b) Explore the hyperparameter tuning results for one of your tuned models. Look for hyperparameter combinations that tied for first place. Test each of these models using train-test split. If no models tied for first place, test your top three models.

In [15]:
# INSTANTIATING a logistic regression model with tuned values
model = RandomForestRegressor( 
                              min_samples_leaf = 26, 
                              max_depth = 9, 
                              criterion = 'absolute_error',
                              random_state = 219)


# FITTING to the TRAINING data
model.fit(x_train, y_train)


# PREDICTING based on the testing set
model.predict(x_test)


# SCORING the results
print(f"""Train Score: {model.score(x_train, y_train).round(decimals = 4)}""")
print(f"""Test Score:  {model.score(x_test, y_test).round(decimals = 4)}""")

# difference between train and test scores
train_score = model.score(x_train, y_train)
test_score = model.score(x_test, y_test)
print(f"""Gap:         {abs(train_score - test_score).round(decimals = 4)}""")


Train Score: 0.8577
Test Score:  0.8342
Gap:         0.0236


In [22]:
def tuning_results(cv_results, n=5):
    """
This function will display the top "n" models from hyperparameter tuning,
based on "rank_test_score".

PARAMETERS
----------
cv_results = results dictionary from the attribute ".cv_results_"
n          = number of models to display
    """
    param_lst = []

    for result in cv_results["params"]:
        result = str(result).replace(":", "=")
        param_lst.append(result[1:-1])


    results_df = pd.DataFrame(data = {
        "Model_Rank" : cv_results["rank_test_score"],
        "Mean_Test_Score" : cv_results["mean_test_score"],
        "SD_Test_Score" : cv_results["std_test_score"],
        "Parameters" : param_lst
    })


    results_df = results_df.sort_values(by = "Model_Rank", axis = 0)
    return results_df.head(n = n)

In [24]:

model_cv.cv_results_

NameError: name 'model_cv' is not defined

In [18]:
# run tuning_results() on the hyperparameter tuning results
tuning_results(cv_results = model_cv.cv_results_, n = 5)

NameError: name 'model_cv' is not defined

In [ ]:
print(f"""
OLS {tuning_results}
""")

<hr style="height:.9px;border:none;color:#333;background-color:#333;" />
<h3>Step 8: (optional) Explore Tuned Hyperparameters</h3><br>
After tuning, it is a great idea to explore any hyperparameters that were markedly different from their default values once tuned. Investigate the purpose of these hyperparameters in <a href="https://scikit-learn.org/stable/index.html">the scikit-learn documentation</a>.